In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule_multigpu
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "8"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton
from ase.optimize import LBFGS

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

from ase.io import read, write

from ase.parallel import world

van Der Waals correction will be unavailable. Please install ased3


In [2]:
dir = '/nh/nest/u/jsmith/scratch/Research/carbon_nano_tube/MD/functionalized_2_justin/11_0/o1/'

fname = 'geometry'

# Molecule file
molfile = dir + fname + '.xyz'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'opt_' +  fname +'.xyz'

dt = 0.5
C = 0.0001 # Optimization convergence

ntdir = '/nh/nest/u/jsmith/scratch/Research/ccsd_extrapolation/final_train_models_3/ani-1x_dft_x8ens/'
cns = ntdir + 'rHCNO-5.2R_16-3.5A_a4-8.params'
sae = ntdir + 'sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 8

In [3]:
tvec = np.loadtxt(dir+'/'+fname+'.tvec',dtype=str)
tvec = np.array(tvec[:,1:],dtype=np.float64)

In [4]:
tvec

array([[18.617822,  0.      ,  0.      ],
       [ 0.      , 18.617822,  0.      ],
       [ 0.      ,  0.      , 21.315   ]])

In [5]:
# Load molecule
mol = read(molfile)
#L=40.0
# mol.set_cell(([[100.0, 0, 0],
#                 [0, 35.0, 0],
#                 [0, 0, 35.0]]))

mol.set_cell((tvec))


mol.set_pbc((True, True, True))
#mol.set_pbc((False, False, False))

# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)
#aens = ensemblemolecule(cns, sae, nnf, Nn, 1)

# Set ANI calculator
mol.set_calculator(ANIENS(aens))

s_time = time.time()
E = mol.get_potential_energy()
F = mol.get_forces()
print('Total time:',time.time()-s_time)

print(E)

spc = mol.get_chemical_symbols()
print(len(spc))

Total time: 0.043035030364990234
-230141.4565420742
221


In [6]:
# Optimize molecule
start_time = time.time()
dyn = LBFGS(mol)
dyn.run(fmax=C,steps=3000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)


hdt.writexyzfile(optfile, pos, spc)

       Step     Time          Energy         fmax
LBFGS:    0 14:11:54  -230141.456542        5.1179
LBFGS:    1 14:11:54  -230142.289388        3.0761
LBFGS:    2 14:11:54  -230142.999087        2.2283
LBFGS:    3 14:11:54  -230143.652140        1.7346
LBFGS:    4 14:11:54  -230144.409761        1.6621
LBFGS:    5 14:11:54  -230145.168536        1.7321
LBFGS:    6 14:11:54  -230145.896536        1.4903
LBFGS:    7 14:11:54  -230146.586027        1.5534
LBFGS:    8 14:11:54  -230147.231621        1.6562
LBFGS:    9 14:11:54  -230147.796561        1.8038
LBFGS:   10 14:11:54  -230148.304636        2.1870
LBFGS:   11 14:11:54  -230148.693746        2.4109
LBFGS:   12 14:11:54  -230148.916562        2.1045
LBFGS:   13 14:11:54  -230149.112529        2.0821
LBFGS:   14 14:11:54  -230149.204110        1.0870
LBFGS:   15 14:11:54  -230149.250937        0.7084
LBFGS:   16 14:11:54  -230149.274433        0.3395
LBFGS:   17 14:11:54  -230149.281488        0.2701
LBFGS:   18 14:11:54  -230149.29

LBFGS:  160 14:11:56  -230149.583189        0.0172
LBFGS:  161 14:11:56  -230149.583338        0.0144
LBFGS:  162 14:11:56  -230149.583468        0.0107
LBFGS:  163 14:11:56  -230149.583591        0.0108
LBFGS:  164 14:11:56  -230149.583691        0.0105
LBFGS:  165 14:11:56  -230149.583772        0.0107
LBFGS:  166 14:11:56  -230149.583863        0.0076
LBFGS:  167 14:11:56  -230149.583939        0.0077
LBFGS:  168 14:11:56  -230149.584021        0.0071
LBFGS:  169 14:11:56  -230149.584099        0.0086
LBFGS:  170 14:11:56  -230149.584151        0.0084
LBFGS:  171 14:11:56  -230149.584249        0.0075
LBFGS:  172 14:11:56  -230149.584307        0.0081
LBFGS:  173 14:11:56  -230149.584403        0.0074
LBFGS:  174 14:11:56  -230149.584445        0.0061
LBFGS:  175 14:11:56  -230149.584512        0.0061
LBFGS:  176 14:11:56  -230149.584555        0.0061
LBFGS:  177 14:11:56  -230149.584618        0.0060
LBFGS:  178 14:11:56  -230149.584658        0.0064
LBFGS:  179 14:11:56  -230149.5

LBFGS:  321 14:11:57  -230149.611308        0.0008
LBFGS:  322 14:11:57  -230149.611311        0.0007
LBFGS:  323 14:11:57  -230149.611311        0.0007
LBFGS:  324 14:11:57  -230149.611322        0.0008
LBFGS:  325 14:11:57  -230149.611306        0.0007
LBFGS:  326 14:11:57  -230149.611319        0.0006
LBFGS:  327 14:11:57  -230149.611304        0.0007
LBFGS:  328 14:11:57  -230149.611324        0.0008
LBFGS:  329 14:11:57  -230149.611308        0.0008
LBFGS:  330 14:11:57  -230149.611329        0.0014
LBFGS:  331 14:11:57  -230149.611317        0.0039
LBFGS:  332 14:11:57  -230149.611332        0.0024
LBFGS:  333 14:11:57  -230149.611331        0.0013
LBFGS:  334 14:11:57  -230149.611330        0.0007
LBFGS:  335 14:11:57  -230149.611320        0.0007
LBFGS:  336 14:11:57  -230149.611315        0.0013
LBFGS:  337 14:11:57  -230149.611326        0.0011
LBFGS:  338 14:11:57  -230149.611332        0.0009
LBFGS:  339 14:11:57  -230149.611325        0.0009
LBFGS:  340 14:11:57  -230149.6

[ANI Total time: 5.160215854644775 seconds]


In [7]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
#dyn = NVTBerendsen(mol, 1 * units.fs, 100.0, taut=1.0*1000*units.fs)
# coefficient to 0.04 atomic units.
dyn = Langevin(mol, dt * units.fs, 1.0 * units.kB, 0.04)

In [8]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  hdt.evtokcal*a.calc.stddev
    Fstddev =  hdt.evtokcal*a.calc.Fstddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + ' ' + str(stddev) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=False)
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f Esig = %.3f Fsig = %.3f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin, stddev,Fstddev))
    
# Set the printer
dyn.attach(storeenergy, interval=50)

In [9]:
dyn.run(2000000)

start_time = time.time()
dyn.set_temperature(300. * units.kB)
dyn.run(2000)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

Step: 50 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=0.093K)  Etot = -1041.400934 Esig = 62.564 Fsig = 0.260
Step: 100 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=0.171K)  Etot = -1041.400912 Esig = 62.579 Fsig = 0.260
Step: 150 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=0.254K)  Etot = -1041.400890 Esig = 62.563 Fsig = 0.260
Step: 200 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=0.317K)  Etot = -1041.400871 Esig = 62.542 Fsig = 0.260
Step: 250 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=0.389K)  Etot = -1041.400857 Esig = 62.524 Fsig = 0.261
Step: 300 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=0.437K)  Etot = -1041.400844 Esig = 62.584 Fsig = 0.260
Step: 350 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=0.502K)  Etot = -1041.400832 Esig = 62.576 Fsig = 0.260
Step: 400 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=0.543K)  Etot = -1041.400815 Esig = 62.567 Fsig = 0.261
Step: 450 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=0.60

Step: 3550 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=1.032K)  Etot = -1041.400688 Esig = 62.647 Fsig = 0.261
Step: 3600 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=1.003K)  Etot = -1041.400691 Esig = 62.615 Fsig = 0.264
Step: 3650 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=1.038K)  Etot = -1041.400690 Esig = 62.583 Fsig = 0.263
Step: 3700 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=1.059K)  Etot = -1041.400680 Esig = 62.612 Fsig = 0.263
Step: 3750 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=1.048K)  Etot = -1041.400686 Esig = 62.647 Fsig = 0.261
Step: 3800 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=1.021K)  Etot = -1041.400694 Esig = 62.646 Fsig = 0.262
Step: 3850 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=0.953K)  Etot = -1041.400690 Esig = 62.611 Fsig = 0.263
Step: 3900 Energy per atom: Epot = -1041.401  Ekin = 0.000 (T=1.002K)  Etot = -1041.400693 Esig = 62.585 Fsig = 0.262
Step: 3950 Energy per atom: Epot = -1041.401  Ekin = 0.0

Step: 7000 Energy per atom: Epot = -1041.365  Ekin = 0.039 (T=303.630K)  Etot = -1041.326223 Esig = 65.710 Fsig = 0.505
[ANI Total time: 16.372895002365112 seconds]
